## ML Foundations - Capstone Project
Theekshana Wijesinghe

### Project: Rank nursery applications

Data source: https://archive.ics.uci.edu/dataset/76/nursery

#### Description

This project is based off of one of the popular datasets in UCI Machine Learning Repository's, known as **Nursery**.


*Nursery Database was derived from a hierarchical decision model originally developed to rank applications for nursery schools.*

The goal of this project is to generate a Machine Learning model for the hierarchical decision model and to predict the rank of the nursery applications.

There are 8 features in the dataset all are **categorical** and the target is a **multiclass** variable. Since all the features are categorical, identifying relation between features and target is a challange.

This problem is **Classification** problem with a multiclass categorization and I will be using Supervised learning approach to solve the problem.


In [ ]:
#Load libraries

import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import OneHotEncoder, LabelBinarizer

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

from sklearn.metrics import RocCurveDisplay, accuracy_score, precision_score, f1_score, recall_score

from sklearn.model_selection import GridSearchCV

from joblib import dump



In [ ]:
#Load data

column_names = ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'class']


data = pd.read_csv('https://github.com/theekshanawj/capstone-project-slasscom/blob/main/dataset/nursery.data?raw=true', names=column_names)

In [ ]:
# See data

print(f"Shape {data.shape}")

data.describe(include='all').transpose()

print(data.dtypes)


In [ ]:
# Data repo mentions no missing value, but let's verify

data.isna().sum()

In [ ]:
# Find duplicates
duplicates = data.loc[data.duplicated()]
duplicates

In [ ]:
# Let's draw bar charts to see data

def drawBarGraph(df, field, ax, title):
  df[field].value_counts().plot(kind='bar', ax=ax, title=title)

fig, axes = plt.subplots(nrows=2, ncols=5)

fig.set_figheight(15)
fig.set_figwidth(15)

idx = 0

for i in range(2):
  for j in range(5):
    if(idx < len(column_names)):
      field = column_names[idx]
      drawBarGraph(data, field, axes[i,j], field)
      idx += 1


#### Correlation between categorical data

[Chi-squared test](https://en.wikipedia.org/wiki/Chi-squared_test) will be approach used here to find the correlation between two categorical variables.

**scipy.stats** library will be used here

If p value is <= 0.05 we can safetly assume that there is a relation between two variables






In [ ]:
#Chi-squared test for feature selection

p_val = []

all_feature_columns = ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health']


def calculate_p_values():
  for i in all_feature_columns:
    crosstab = pd.crosstab(data['class'], data[i])
    c, p, dof, expected = chi2_contingency(crosstab)
    p_val.append(np.abs(p))

    plt.figure(figsize=(6,2))
    sns.heatmap(crosstab, annot=True, cmap="YlGnBu")


calculate_p_values()

p_values = pd.DataFrame({ 'p_value': p_val, 'field': all_feature_columns }, index=all_feature_columns)

p_values.plot(kind='bar', title='Probability value of Chi-Square test')

filtered_field = p_values.loc[p_values['p_value'] <= 0.05 ]

filtered_field


In [ ]:
# See the uniques of target variable and assign numerical value
# This is categorical and ordinal in a way

data['class'].unique()

print(data['class'].value_counts())


class_map =  {
    'not_recom': 0,
    'recommend': 1,
    'very_recom': 2,
    'priority': 3,
    'spec_prior': 4
}


y = data['class'].replace(class_map)

y.sample(10)


In [ ]:
# Record with recommend value as the target is very small, this could be problametic
# First I will keep this in the dataset and see the results and then delete it to see how this will impact the model accuracy


data.loc[data['class'] == 'recommend']


# Learnt lesson:  We get 'UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.' when running SVC
# Therefore it is required to remove recommend class

In [ ]:
# Let's model using logistic regression
# One hot encoding is done using sklearn pipeline, this avoid tedious operations using Pandas
# This way we can easily use the same when we try to predict future values


# Define the type of model
model = LogisticRegression(max_iter=500)

# Let's start with the features that showed p = 0 for chi-squared test
feature_columns = ['has_nurs', 'health']

# Define the transfomers
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, feature_columns), #All the feature columns are categorical
    ]
)


X = data[feature_columns]

# Transfomr y into categorical column, this is same as above

y = data['class'].replace(class_map)

pipe = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", model)]
)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

pipe.fit(X_train, y_train)

predict = pipe.predict(X_test)

In [ ]:
#Evaluate results

print(predict.shape)

result = pd.DataFrame({ 'y_actual': y_test, 'y_pred': predict })

result.plot(kind='scatter', x='y_actual', y='y_pred')

print(result)

pd.crosstab(result['y_actual'], result['y_pred'])

In [ ]:
# Let's evaluate the model metrices

print(f"Accuracy: {accuracy_score(result['y_actual'], result['y_pred'])}")

print(f"Prcision: {precision_score(result['y_actual'], result['y_pred'], average='weighted')}")

print(f"Recall: {recall_score(result['y_actual'], result['y_pred'], average='weighted')}")

print(f"F1 score: {f1_score(result['y_actual'], result['y_pred'], average='weighted')}")


### Multiclass Receiver Operating Characteristic (ROC)

Reference: https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html

In [ ]:
# ROC curve

def draw_roc_curves(pipe, X_test, y_train, y_test, model_name):

  y_score = pipe.predict_proba(X_test)

  label_binarizer = LabelBinarizer().fit(y_train)

  y_onehot_test = label_binarizer.transform(y_test)

  for key in class_map.keys():
    interested_label_id = class_map[key]

    class_ids = np.flatnonzero([class_map[key] == label_binarizer.classes_])

    if len(class_ids) > 0:
      interested_label_id = class_ids[0]
      RocCurveDisplay.from_predictions(
          y_onehot_test[:, interested_label_id],
          y_score[:, interested_label_id],
          name=f"{key} against others",
          color="darkorange",
      )
      plt.axis("square")
      plt.xlabel("False Positive Rate")
      plt.ylabel("True Positive Rate")
      plt.title(f"{key} vs Rest for {model_name}")
      plt.legend()
      plt.show()





draw_roc_curves(pipe, X_test, y_train, y_test, "LG")


In [ ]:
# Above model is based of two features let's change the features and see if it impacts the metrices

# Define the type of model
model = LogisticRegression(max_iter=500)

features_1 = ['has_nurs', 'health']
features_2 = ['parents', 'has_nurs', 'children', 'housing', 'social', 'health'] # Without form and finance that showed high p value in above
features_3 = ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health']

features_list = [features_1, features_2, features_3]


def get_pipeline(model, features):

    categorical_transformer = Pipeline(
        steps=[
            ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ]
    )

    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, features)
        ]
    )

    return Pipeline(
        steps=[("preprocessor", preprocessor), ("classifier", model)]
    )



def get_features(features):
  return data[features]


def get_target():
  return data['class'].replace(class_map)


def train_model(pipe, X_train, y_train):
      pipe.fit(X_train, y_train)

def calculate_accuracy(result):
  return round(accuracy_score(result['y_actual'], result['y_pred']), 3)

def calculate_precision(result):
  return round(precision_score(result['y_actual'], result['y_pred'], average='weighted'), 3)

def calculate_f1_score(result):
  return round(f1_score(result['y_actual'], result['y_pred'], average='weighted'), 3)

def calculate_recall(result):
  return round(recall_score(result['y_actual'], result['y_pred'], average='weighted'), 3)


def evaluate_results(features, y_test, predict):

    feature_name = f"{features}"

    print(f"Features: {feature_name}")

    result = pd.DataFrame({ 'y_actual': y_test, 'y_pred': predict })

    result.plot(kind='scatter', x='y_actual', y='y_pred', title=feature_name)

    pd.crosstab(result['y_actual'], result['y_pred'])

    print(f"Accuracy: {calculate_accuracy(result)}")

    print(f"Prcision: {calculate_precision(result)}")

    print(f"Recall: {calculate_recall(result)}")

    print(f"F1 score: {calculate_f1_score(result)}")


# Extract everything to a function
def evaluate_model_with_variable_features(model, features):

    pipe = get_pipeline(model, features)

    X = get_features(features)
    y = get_target()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=30)

    train_model(pipe, X_train, y_train)

    predict = pipe.predict(X_test)

    evaluate_results(features, y_test, predict)


for feature in features_list:
  evaluate_model_with_variable_features(model, feature)


# We can conclude that highest accuracy, precision and f1 score is with all the features included

In [ ]:
# Class 'recommend' is not available in the results and we saw only two records were available in the dataset
# Let's remove those two and see what happens

print(data.loc[data['class'] == 'recommend'])

data.drop(index=[0,3], inplace=True)

print(data.loc[data['class'] == 'recommend'])

# New class map
class_map =  {
    'not_recom': 0,
    'very_recom': 1,
    'priority': 2,
    'spec_prior': 3
}


In [ ]:
#Let's again evaluate the model

evaluate_model_with_variable_features(model, ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health']) # All the features are included


In [ ]:
# Let's now use other models along with the regression model

#Models, here models are used with default params except for LogisticRegression to identify the best model to optimise further with params
models = [LogisticRegression(max_iter=500),
            RandomForestClassifier(),
              KNeighborsClassifier(),
                svm.SVC(probability=True)] # probability=True; This is to draw the ROC curve

feature_columns = ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health']


def get_model_eval_score(result_df):
  return {
      'accuracy': calculate_accuracy(result_df),
      'precision': calculate_accuracy(result_df),
      'recall': calculate_recall(result_df),
      'f1_score': calculate_f1_score(result_df)
  }

def get_model_name(model):
  return str(model).split('(')[0]

def evaluate_model(model):

  pipe = get_pipeline(model, feature_columns)

  X = get_features(feature_columns)
  y = get_target()

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=30)

  train_model(pipe, X_train, y_train)

  predict = pipe.predict(X_test)

  result = pd.DataFrame({'y_actual': y_test, 'y_pred': predict })

  # can be used to draw roc curves, uncomment to see results
  # draw_roc_curves(pipe, X_test, y_train, y_test, get_model_name(model))

  return get_model_eval_score(result)



model_evals = list(map(evaluate_model, models))

mode_names = list(map(get_model_name, models))


def get_eval_df():
  df_object = {}
  for idx, model in enumerate(mode_names):
    df_object[model] = model_evals[idx]

  return df_object

eval_object = get_eval_df()

pd.DataFrame(get_eval_df(), index=['accuracy', 'precision', 'recall', 'f1_score']).plot(kind='bar')

eval_object


In [ ]:
# From above metric table, we can clearly see Support Vector Classification has the best metrices and RandomForestClassifier coming close

# Now let'see if we can improve the model using GridSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

svc = svm.SVC(probability=True)

model = GridSearchCV(svc, parameters, scoring='f1_weighted', cv=5)

evaluate_model(model)

print(model.best_score_)
print(model.best_params_)
print(model.best_estimator_)


In [ ]:
#Finally let's save the model


pipe = get_pipeline(model, feature_columns)

X = get_features(feature_columns)
y = get_target()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=30)

train_model(pipe, X_train, y_train)

dump(pipe, 'svc_pipeline.joblib')
